In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import pickle




from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.impute import KNNImputer



In [97]:
df = pd.read_csv("Sri Lankan CKD Dataset.csv")

In [98]:
a= {"M":1,"F":0}

df["gender"]=df["gender"].map(a)

In [99]:
df = df.drop("cru",axis =1)
df = df.drop("ua",axis =1)
df = df.drop("iron",axis =1)

In [100]:
missing_values_per_row = df.isnull().sum(axis=1)

print("Number of Missing Values per Row:")
print(missing_values_per_row)

Number of Missing Values per Row:
0      6
1      2
2      1
3      2
4      2
      ..
401    4
402    0
403    1
404    4
405    1
Length: 406, dtype: int64


In [101]:
threshold = 10
filtered_data = df[missing_values_per_row <= threshold]

print("\nFiltered DataFrame:")
print(filtered_data)
filtered_data.shape



Filtered DataFrame:
      id   age  weight  gender       cr    al    gl     na    ca    k     cl  \
0      1  72.0   52.00       1   249.43  34.9   NaN  142.0  2.43  5.0  113.0   
1      2  76.0   52.00       1   216.05  41.3   NaN  145.0  2.40  4.7  114.0   
2      3  60.0   80.40       1   155.40  33.7   NaN  135.0  2.00  4.2  103.0   
3      4  65.0   53.40       0   127.50  41.8   NaN  138.0  2.80  4.4  100.0   
4      5  59.0   60.40       1   172.71  39.7   NaN  128.0  2.20  3.6   91.0   
..   ...   ...     ...     ...      ...   ...   ...    ...   ...  ...    ...   
401  402  48.0   48.00       1  1167.62   NaN   NaN  134.0   NaN  4.1   94.0   
402  403  46.0   55.00       1   211.20  37.2  37.9  144.0  2.40  4.3  105.0   
403  404  55.0   54.85       0   164.43  22.2  48.9  129.0   NaN  3.9   97.0   
404  405  84.0   43.90       1   501.71   NaN   NaN  135.0   NaN  5.7   99.0   
405  406  38.0   60.50       1   880.64  38.5  32.1  127.0   NaN  4.1   99.0   

       pr   wbc   

(395, 17)

In [102]:
df = df[['cr','age','weight','al','gender','na','gl','ca','cl','k','class']].copy()


In [103]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=14)

# Perform the imputation
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df_imputed.head()

,cr,age,weight,al,gender,na,gl,ca,cl,k,class
0,249.43,72.0,52.0,34.9,1.0,142.0,35.721429,2.43,113.0,5.0,1.0
1,216.05,76.0,52.0,41.3,1.0,145.0,36.300000,2.40,114.0,4.7,1.0
2,155.40,60.0,80.4,33.7,1.0,135.0,34.230000,2.00,103.0,4.2,1.0
3,127.50,65.0,53.4,41.8,0.0,138.0,33.480000,2.80,100.0,4.4,1.0
4,172.71,59.0,60.4,39.7,1.0,128.0,37.908571,2.20,91.0,3.6,1.0


In [104]:
df_imputed = df_imputed.round(1)
X = df_imputed.drop(columns='class')
y = df_imputed['class']

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the XGBoost Classifier
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)

# Step 4: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")



Model Accuracy: 0.95


In [106]:
X_train

,cr,age,weight,al,gender,na,gl,ca,cl,k
395,136.2,51.0,48.6,34.8,0.0,142.0,31.7,2.3,107.0,3.5
3,127.5,65.0,53.4,41.8,0.0,138.0,33.5,2.8,100.0,4.4
18,247.7,68.0,58.0,37.9,1.0,133.0,35.7,2.2,97.0,2.3
131,1071.6,52.0,56.9,35.4,1.0,134.0,31.8,2.5,95.0,5.7
118,304.1,52.0,77.0,49.0,0.0,140.0,33.5,2.3,93.0,4.6
...,...,...,...,...,...,...,...,...,...,...
71,408.1,54.0,64.0,37.1,0.0,136.4,33.5,2.1,95.2,4.0
106,133.3,32.0,57.0,38.5,1.0,138.0,36.1,2.2,102.0,4.1
270,303.1,80.0,66.8,36.5,1.0,135.0,30.6,2.2,100.1,4.1
348,70.7,52.0,68.3,36.0,0.0,135.0,35.5,2.5,99.5,3.7


In [107]:
X_test.round(2)

,cr,age,weight,al,gender,na,gl,ca,cl,k
70,716.2,60.0,84.0,38.8,1.0,121.0,31.0,2.1,77.0,4.5
218,88.4,30.0,72.6,38.0,1.0,147.0,36.4,2.4,99.1,3.8
258,61.9,45.0,56.3,36.1,0.0,141.0,43.6,2.5,102.0,4.2
33,137.1,68.0,60.0,40.0,1.0,136.0,33.2,2.2,98.0,3.9
42,253.8,49.0,60.0,37.2,1.0,125.0,35.7,2.2,90.0,4.3
...,...,...,...,...,...,...,...,...,...,...
90,126.5,54.0,75.0,40.7,1.0,139.0,33.0,2.4,105.0,4.3
378,1141.8,35.0,48.0,42.8,1.0,141.0,31.8,2.2,95.0,7.2
340,70.7,48.0,42.5,42.0,1.0,137.0,32.6,2.5,102.2,5.0
375,220.0,34.0,47.0,32.9,1.0,134.0,20.2,2.5,98.3,3.5


In [118]:
x_ = np.array([716.230,60.0, 84.0,38.800000,	1.0,121.0,31.042857,2.100000,77.000000,4.5])
x_ = x_.reshape(1, -1)

# single_instance = np.array([4.5, 2.3, 1.3, 0.2, 3.1])

# # Reshape it into a 2D array
# single_instance_reshaped = single_instance.reshape(1, -1)

In [119]:
import pickle

# Load the saved XGBoost model
with open("model_folder/xgboost_ckd_model.pkl", "rb") as file:
    m = pickle.load(file)

In [120]:
y_ = m.predict(x_)
y_

array([1])

In [117]:
y_

array([1])

In [111]:
y_test

70     1.0
218    0.0
258    0.0
33     1.0
42     1.0
      ... 
90     1.0
378    1.0
340    0.0
375    1.0
356    0.0
Name: class, Length: 82, dtype: float64

In [112]:
import os
folder_name = "model_folder"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")

# Step 6: Save the model in the new folder
model_path = os.path.join(folder_name, 'xgboost_ckd_model_no_scr.pkl')
with open(model_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved as {model_path}")

Model saved as model_folder\xgboost_ckd_model_no_scr.pkl
